## CartPole
### Reinforcement Learning (DQN) Tutorial text
Link: https://h-huang.github.io/tutorials/intermediate/reinforcement_q_learning.html

#### Imports

In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch as pt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T



env = gym.make('CartPole-v0').unwrapped

# Set up matplotlib
if is_ipython := 'inline' in matplotlib.get_backend():
    from IPython import display

plt.ion()

# If GPU is to be used
device = pt.device('cuda' if pt.cuda.is_available() else 'cpu')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


#### Replay Memory

experience replay memory for training our DQN. It stores the transitions that the agent observes, which can be reused later. By sampeling from it randomly the transitions that buld up a batch are decorrelated. It has been shown that this greatly stabilizes and improves the training procedure.

Two classes in use:
* Transition - a named tuple represented by a single transition in our environment. It maps (state, action) to this paired (next_state, reward) result
* ReplayMemory - a cyclic buffer of bounded size that holds the transitions observed recently. It also implements a .sample() method which can be used to select a random batch of transitions for training.

In [2]:
Transition = namedtuple('Transition', 
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    '''ReplayMemory stores the transitions that the agent observes, allowing us to reuse this data later. 
    
    By sampling from it randomly, the transitions that build up a batch are decorrelated.'''
    def __init__(self, capacity) -> None:
        self.memory = deque([], maxlen=capacity)
    
    def push(self, *args):
        '''Saves a transition'''
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### DQN algorithm

Our environment is deterministic, so all equations presented here are also formulated deterministically for the sake of simplicity. In the reinforcement learning literature, they would also contain expectations over stochastic transitions in the environment (randomness with a pattern).

Our aim will be to train a policy that tries to maximize the discounted, cumulative reward 
R_t0 = sum(discountRate * r_t)

The main idea behind Q-learning is that if we have a function Q*: State*Action -> R, That couldtell us what our return would be if we were to take an action in a given state, then we could easily construct a policy that maximizes or rewards:

pi*(s) = argmax Q*(s,a)

However, we don't know everything in the world, so we dont have access to Q*, but we can approximate this. Therefore our updated training rule we can use this Q function

Q^pi(s,a) = r+discount*Q^pi(s', pi(s'))

To minimize the error we can use Huber Loss: https://en.wikipedia.org/wiki/Huber_loss
Which acts like the mean sqeared error, when the error is small, but like the mean absolute error when its large making this method robust:

Q-net
Our model will be feed forward neural network that takes in the difference between the current and previous screen patches. It has two outputs, representing Q(s, left) or Q(s, right), where right or left is the action and s is the input to the network. 



In [3]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))


#### Input extraction

The code below are utilities to extract and process rendered images for the env. It uses pt to compose image transforms. Once you run the cell it will display an example patch that is extracted.

In [10]:
# T stands for pt.transforms
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.BICUBIC),
                    T.ToTensor()])

def get_cart_location(screen_width):
    world_width = env.x_threshold*2
    scale = screen_width/world_width
    return int(env.state[0] * scale + screen_width/2)

def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = pt.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0).to(device)

env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()

error: display Surface quit

### Training

#### Hyperparams and utilities
This cell instantiates our model and its optimizer and defines some utilities:

* select_action - selects action according to epsilon greedy policy. The probability of choosing a random action will start at EPS_Start and move towards EPS_END where EPS_DECAY is the rate of decay. This creates a gradual transformation from exploration to exploitation
* plot_durations - a helper for plotting the duration of episodes, along with the averate for the last 100 episodes (The measure used in the official evaluations). The plot underneath the cell containing the main training loop and will update every episode.



In [5]:
# Params
# BATCH_SIZE is the number of transitions sampled from the replay buffer
BATCH_SIZE = 128

# GAMMA is the discount factor as mentioned in the previous section
GAMMA = 0.99

# EPS_START is the starting value of epsilon
EPS_START = 0.9

# EPS_END is the final value of epsilon
EPS_END = 0.05

# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
EPS_DECAY = 200

#
TARGET_UPDATE = 10

# ------------------------

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
init_screen = get_screen()
_, _, screen_height, screen_width = init_screen.shape


# Get num of actions from gym action space
n_actions = env.action_space.n # left or right


policy_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net = DQN(screen_height, screen_width, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.AdamW(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with pt.no_grad(): # basically similar to inference_mode
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return pt.tensor([[random.randrange(n_actions)]], device=device, dtype=pt.long)

episode_durations = []

# plot_durations - a helper for plotting the durations of episodes
# along with an average over the last 100 episodes (the measure used in the official evaluations). The plot will be underneath the cell containing the main training loop, and will update after every episode.

def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = pt.tensor(episode_durations, dtype=pt.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = pt.cat((pt.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        plt.show()

### Training loop

Info: 
* V(st+1) = Value of the next state 
* Q(st,at) = Q_value of state and action
* Episode = a set amount of interactions between agent and env

finally, the code for the training of the model

Here, you can find an optimize_model function that performs a single step of the optimization. 
1. samples a batch
2. concatenates all the tensors into a single one
3. Computes Q(st,at) and V(st+1)=maxaQ(st+1,a)
4. Combines them into our loss. 

By definition we set V(s)=0 if s is a terminal state. 

We also use a target network to compute V(st+1) for added stability. The target network has its weights kept frozen most of the time, but is updated with the policy network’s weights every so often. This is usually a set number of steps but we shall use episodes for simplicity.

In [6]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = pt.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=pt.bool)
    non_final_next_states = pt.cat([s for s in batch.next_state
                                                if s is not None])
    
    state_batch = pt.cat(batch.state, 0)
    action_batch = pt.cat(batch.action, 0)
    reward_batch = pt.cat(batch.reward, 0)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each state according to policy_net

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = pt.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

 

Main training loop

In [9]:
num_episodes = 50
for i_episode in range(num_episodes):
    # initialize the env and state
    env.reset()
    last_screen = get_screen()
    current_screen = get_screen()
    state = current_screen - last_screen
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = pt.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None
        
        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy net)
        optimize_model()
        if done:
            episode_durations.append(t+1)
            plot_durations()
            break

        # Update the target network, copying all the weights and biases in DQN
        if i_episode % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())
print('Complete')
env.render()
env.close()
plt.ioff()
plt.show()

error: display Surface quit